<a href="https://colab.research.google.com/github/Rafaelfyh/Asistencia_delivery/blob/main/Projeto_FraudGuard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install azure-ai-formrecognizer azure-ai-textanalytics azure-cognitiveservices-vision-computervision


In [ ]:
from azure.ai.formrecognizer import FormRecognizerClient
from azure.ai.textanalytics import TextAnalyticsClient
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Configurações iniciais para credenciais e endpoints
form_recognizer_endpoint = "https://<seu-form-recognizer>.cognitiveservices.azure.com/"
form_recognizer_key = "<sua-chave-form-recognizer>"
text_analytics_endpoint = "https://<seu-text-analytics>.cognitiveservices.azure.com/"
text_analytics_key = "<sua-chave-text-analytics>"
computer_vision_endpoint = "https://<seu-computer-vision>.cognitiveservices.azure.com/"
computer_vision_key = "<sua-chave-computer-vision>"

# Instanciação dos clientes do Azure
form_recognizer_client = FormRecognizerClient(form_recognizer_endpoint, AzureKeyCredential(form_recognizer_key))
text_analytics_client = TextAnalyticsClient(endpoint=text_analytics_endpoint, credential=AzureKeyCredential(text_analytics_key))
computer_vision_client = ComputerVisionClient(computer_vision_endpoint, AzureKeyCredential(computer_vision_key))


#1. Extração de Dados com o Form Recognizer

In [ ]:
def extract_data_from_document(file_path):
    # Abrir o documento para análise
    with open(file_path, "rb") as file:
        poller = form_recognizer_client.begin_recognize_content(file)
        result = poller.result()

    # Extração de textos e coordenadas dos campos
    for page in result:
        print(f"Página {page.page_number}:")
        for table in page.tables:
            for cell in table.cells:
                print(f"Texto: {cell.text} (linha: {cell.row_index}, coluna: {cell.column_index})")

    return result


#2. Análise de Texto com Text Analytics

In [ ]:
def analyze_text(text):
    documents = [text]
    response = text_analytics_client.analyze_sentiment(documents=documents)[0]
    print("Análise de Sentimento:")
    print(f"Sentimento Geral: {response.sentiment}")

    for idx, sentence in enumerate(response.sentences):
        print(f"Frase {idx + 1}: Sentimento: {sentence.sentiment}, Pontuação: {sentence.confidence_scores}")

    return response


#3. Análise de Imagem e Autenticidade com Computer Vision

In [ ]:
def analyze_image(file_path):
    # Abre a imagem para análise
    with open(file_path, "rb") as image:
        analyze_image_result = computer_vision_client.analyze_image_in_stream(image, visual_features=["Description", "Tags"])

    print("Descrição da imagem:")
    for caption in analyze_image_result.description.captions:
        print(f"Descrição: {caption.text}, Confiança: {caption.confidence:.2f}")

    print("Tags da imagem:")
    for tag in analyze_image_result.tags:
        print(f"Tag: {tag.name}, Confiança: {tag.confidence:.2f}")

    return analyze_image_result


#4. Identificação de Fraudes com Modelo de Machine Learning (Simulação)

In [ ]:
import joblib  # Biblioteca para carregar modelos locais

def fraud_detection(model_path, extracted_data):
    # Carregar modelo treinado (supondo que temos um modelo local)
    model = joblib.load(model_path)

    # Processar os dados extraídos (ajuste conforme a necessidade do modelo)
    features = [extracted_data]  # Transforme os dados em recursos de entrada

    # Predição de fraude
    prediction = model.predict(features)
    print("Possível Fraude Detectada!" if prediction == 1 else "Documento Aprovado.")

    return prediction


#5. Execução do Pipeline Completo


In [ ]:
def process_document(file_path, model_path):
    # Passo 1: Extração de dados do documento
    extracted_data = extract_data_from_document(file_path)

    # Passo 2: Análise textual (exemplo simples de uma string extraída)
    sample_text = "Conteúdo extraído para análise de texto."
    analyze_text(sample_text)

    # Passo 3: Análise de imagem
    analyze_image(file_path)

    # Passo 4: Detecção de fraude usando modelo de ML
    fraud_detection(model_path, extracted_data)

# Exemplo de uso
file_path = "caminho/para/o/documento.pdf"
model_path = "caminho/para/modelo_fraude.joblib"
process_document(file_path, model_path)


#Observações Finais
*Modelos de Machine Learning: A detecção de fraudes é uma parte complexa que* *normalmente envolve dados históricos e treinamento extensivo no Azure ML* *Studio. Este script assume que um modelo treinado já está disponível em* *formato .joblib.*
*Tratamento de Exceções e Logs: Devemos incluir tratamento de* *exceções e logging, especialmente ao lidar com serviços externos.*
*Privacidade e Segurança: Deve-se conexões seguras e controlar o acesso aos documentos.